## RTS Data Analyst take-home assignment

🔍 Step 1: Understand the Objective
You need to:
- Recommend volume per content (i.e., how much to produce) for each of 5 themes: info, sport, musique, societe, humour
- Help business understand the function each theme serves:
acquisition, retention, or loyalty
- Communicate this with clear insights and visuals

🧹 Step 2: Data Exploration & Cleaning
📁 Files Overview
You have:

1. Mesures_contenu_volume_audio_à_commander.csv: performance metrics for segments (episodes)
2. Correspondance_show_segment_tag.csv: mapping from episodes to tags (themes)

Actions:
- Load both CSVs
- Join them on segment ID
- Filter to:
  - Date from Jan 2025 onward (for tags to be valid)
  - Platform = "rts.ch" (only this has theme data)
  - Keep only the 5 themes of interest via assigned tags

📊 Step 3: Define Key Metrics per Theme
For each of the 5 themes, calculate:

# of segments

- Total play duration (overall consumption)
- Average play duration per segment
- Visitors: how many users reached the content
- New visit rate: to estimate acquisition
- Returning visits and bounces: for retention vs. bounce
- Entries and Exits: to see if the segment starts or ends visits

This will let you profile each theme:
- Acquisition = High entries + High new visit rate
- Retention = Low bounce + Long play duration
- Loyalty = High returning visitors + Low exit

### 0. Librairies

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

### 1. Data Ingestion

In [29]:
# Load the CSV file
path_volume = "../data/Mesures_contenu_volume_audio_à_commander.csv"
df_volume = pd.read_csv(path_volume, sep=';', encoding='utf-8')

# Show first few rows of each for context
df_volume

,Segment ID,Segment,Show ID,Show,Publication Date,App/Site Name,Device Class,Segment Length,Media Views,Avg Play Duration,Visitors,New Visit Rate %,Entries,Exits,Returning Visits,Bounces,Total Play Duration
0,14897825,Le Suisse Nemo triomphe à lEurovision avec sa ...,2031524,Le Journal horaire,12.05.2024,rts.ch,Smartphone,1234,20762,00:05:19,18877,"84,56%",9770,13135,3428,5181,94:50:23
1,15102359,Une trombe sest formée au-dessus du lac Léman,2031524,Le Journal horaire,18.08.2024,rts.ch,Smartphone,586,14703,00:03:27,13381,"53,30%",9889,11505,6458,6798,108:13:53
2,14572281,De Genève à Zurich: un périple sanglant en Hel...,14546712,Crimes suisses,05.01.2024,rts-app-play,Smartphone,3490,7327,00:24:41,4124,"2,49%",1527,1928,6594,602,2601:23:11
3,14689374,Prise dotages dans un train près dYverdon: les...,8849020,La Matinale,09.02.2024,rts.ch,Smartphone,1500,7560,00:06:25,7934,"71,32%",4370,4993,2671,2729,151:43:36
4,359fc205-7470-38e0-b393-3b4a2e429508,Pourquoi les couples se séparent,6067786,Tribu,07.04.2025,rts.ch,Smartphone,1956,7201,00:08:34,7147,"43,80%",6741,3901,4808,4016,851:19:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277581,8d3ad86d-97e1-372b-bc60-8e0f58643b37,"Face au défi climatique, Neuchâtel fait un app...",1423859,Le 12h30,12.05.2025,rts-app-sport,Smartphone,564,524,00:01:01,90,"0,00%",103,140,23,726,00:04:16
277582,96015a33-f517-3cf8-bcda-c9658dd6c844,En Douceur,14570123,En Douceur,12.05.2025,rts.ch,Smartphone,4677,451,00:04:25,772,"103,00%",141,802,695,687,00:04:11
277583,0267bc07-2c73-327c-9f5b-f692289ed9d2,Le Suisse mort en Ukraine était un Lausannois ...,1784426,Forum,28.03.2025,rts-app-sport,Smartphone,814,438,00:05:24,989,"0,00%",476,772,859,92,00:00:12
277584,41568641-62b4-3596-99ce-3b8bf4d09ad8,Helveticus,12027724,Léchappée,28.03.2025,rts.ch,Smartphone,1150,512,00:05:02,289,"103,00%",1222,1055,82,889,00:00:14


In [3]:
df_volume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277586 entries, 0 to 277585
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Segment ID           277586 non-null  object
 1   Segment              277586 non-null  object
 2   Show ID              277467 non-null  object
 3   Show                 277467 non-null  object
 4   Publication Date     277467 non-null  object
 5   App/Site Name        277467 non-null  object
 6   Device Class         277467 non-null  object
 7   Segment Length       277586 non-null  int64 
 8   Media Views          277586 non-null  int64 
 9   Avg Play Duration    277586 non-null  object
 10  Visitors             277586 non-null  int64 
 11  New Visit Rate %     277586 non-null  object
 12  Entries              277586 non-null  int64 
 13  Exits                277586 non-null  int64 
 14  Returning Visits     277586 non-null  int64 
 15  Bounces              277586 non-nu

In [4]:
df_volume.describe()

,Segment Length,Media Views,Visitors,Entries,Exits,Returning Visits,Bounces
count,277586.000000,277586.000000,277586.000000,277586.00000,277586.000000,277586.000000,277586.000000
mean,2266.756205,328.131336,667.504734,630.95284,639.351001,661.112099,624.488497
std,2800.624729,215.137803,375.419407,361.20359,362.775949,373.208065,358.033180
min,6.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000
25%,783.000000,178.000000,353.000000,321.00000,328.000000,346.000000,316.000000
50%,1310.000000,319.000000,666.000000,632.00000,639.000000,659.000000,623.000000
75%,2131.000000,457.000000,976.000000,941.00000,949.000000,969.000000,934.000000
max,22957.000000,20762.000000,18877.000000,9889.00000,13135.000000,6594.000000,6798.000000


In [5]:
df_volume.isnull().sum()

Segment ID               0
Segment                  0
Show ID                119
Show                   119
Publication Date       119
App/Site Name          119
Device Class           119
Segment Length           0
Media Views              0
Avg Play Duration        0
Visitors                 0
New Visit Rate %         0
Entries                  0
Exits                    0
Returning Visits         0
Bounces                  0
Total Play Duration      0
dtype: int64

In [27]:
# Load the CSV file
path_tags = "../data/Correspondance_show_segment_tag.csv"
df_tags = pd.read_csv(path_tags, sep=';', encoding='utf-8')

# Show few rows for context
df_tags

,Segment ID,Show,Show ID,Assigned Tags
0,14897825,Le Journal horaire,2031524,-
1,15102359,Le Journal horaire,2031524,-
2,15112045,Crimes suisses,14546712,-
3,14689374,La Matinale,8849020,-
4,15126915,Vertigo,4197907,-
...,...,...,...,...
107797,14818255,Forum,1784426,media_radio:media_radio_media_radio:info_rts_i...
107798,14851148,Forum,1784426,media_radio:la-1ere_rts_info:rts_info_media_tv...
107799,15228940,Forum,1784426,media_radio:media_radio_rts_info:monde_rts_inf...
107800,14845847,Forum,1784426,media_radio:media_radio_media_radio:info_rts_i...


In [21]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107802 entries, 0 to 107801
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Segment ID     107802 non-null  object
 1   Show           107802 non-null  object
 2   Show ID        107802 non-null  object
 3   Assigned Tags  107802 non-null  object
dtypes: object(4)
memory usage: 3.3+ MB


In [22]:
df_tags.describe()

,Segment ID,Show,Show ID,Assigned Tags
count,107802,107802,107802,107802
unique,80731,491,470,2360
top,15447445,Le Journal horaire,2031524,-
freq,9,18894,18971,59253


In [23]:
df_tags.isnull().sum()

Segment ID       0
Show             0
Show ID          0
Assigned Tags    0
dtype: int64

### 2. Data Cleaning

In [ ]:
df_volume1 = df_volume.dropna().reset_index()
df_volume1

,Segment ID,Segment,Show ID,Show,Publication Date,App/Site Name,Device Class,Segment Length,Media Views,Avg Play Duration,Visitors,New Visit Rate %,Entries,Exits,Returning Visits,Bounces,Total Play Duration
0,14897825,Le Suisse Nemo triomphe à lEurovision avec sa ...,2031524,Le Journal horaire,12.05.2024,rts.ch,Smartphone,1234,20762,00:05:19,18877,"84,56%",9770,13135,3428,5181,94:50:23
1,15102359,Une trombe sest formée au-dessus du lac Léman,2031524,Le Journal horaire,18.08.2024,rts.ch,Smartphone,586,14703,00:03:27,13381,"53,30%",9889,11505,6458,6798,108:13:53
2,14572281,De Genève à Zurich: un périple sanglant en Hel...,14546712,Crimes suisses,05.01.2024,rts-app-play,Smartphone,3490,7327,00:24:41,4124,"2,49%",1527,1928,6594,602,2601:23:11
3,14689374,Prise dotages dans un train près dYverdon: les...,8849020,La Matinale,09.02.2024,rts.ch,Smartphone,1500,7560,00:06:25,7934,"71,32%",4370,4993,2671,2729,151:43:36
4,359fc205-7470-38e0-b393-3b4a2e429508,Pourquoi les couples se séparent,6067786,Tribu,07.04.2025,rts.ch,Smartphone,1956,7201,00:08:34,7147,"43,80%",6741,3901,4808,4016,851:19:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277581,8d3ad86d-97e1-372b-bc60-8e0f58643b37,"Face au défi climatique, Neuchâtel fait un app...",1423859,Le 12h30,12.05.2025,rts-app-sport,Smartphone,564,524,00:01:01,90,"0,00%",103,140,23,726,00:04:16
277582,96015a33-f517-3cf8-bcda-c9658dd6c844,En Douceur,14570123,En Douceur,12.05.2025,rts.ch,Smartphone,4677,451,00:04:25,772,"103,00%",141,802,695,687,00:04:11
277583,0267bc07-2c73-327c-9f5b-f692289ed9d2,Le Suisse mort en Ukraine était un Lausannois ...,1784426,Forum,28.03.2025,rts-app-sport,Smartphone,814,438,00:05:24,989,"0,00%",476,772,859,92,00:00:12
277584,41568641-62b4-3596-99ce-3b8bf4d09ad8,Helveticus,12027724,Léchappée,28.03.2025,rts.ch,Smartphone,1150,512,00:05:02,289,"103,00%",1222,1055,82,889,00:00:14


### 3. Data Transformation

### 4. Feature Engineering